In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
from include import *
from PIL import Image
import PIL
import pywt
import numpy as np
import torch
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from sklearn import linear_model
from scipy.fftpack import dct
from scipy.fftpack import idct
from scipy import io as sio 
import time



GPU = True
if GPU == True:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    dtype = torch.cuda.FloatTensor
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print("num GPUs",torch.cuda.device_count())
    device = 'cuda'
    if torch.cuda.device_count()==0:
        dtype = torch.FloatTensor
        device = 'cpu'
else:
    dtype = torch.FloatTensor
    device = 'cpu'


# 1. Load a test image from a dataset (now : CelebA 128x128)

In [ ]:
#dataset = 'mnist' # 'mnist' or 'celeba'
dataset = 'celeba'
path = './test_data/' + dataset + '/' 
img_name = dataset + '_128x128' # 1-5 (for celeba), 1-6 (for mnist)
img_path = path + img_name + ".jpg"
img_pil = Image.open(img_path)
#if dataset == 'celeba':
#    img_pil = img_pil.crop((60,80+20,60+64,80+84)) #crop to 3 x 64 x 64
img_np = pil_to_np(img_pil)
print('Dimensions of input image:', img_np.shape)
img_np = img_np / np.max(img_np)

#img_np = (img_np- 1/2) *2 #convert to [-1,1] 

img_np_orig = 1*img_np

if dataset == 'celeba':
    plt.imshow(img_np.transpose(1,2,0))
else:
    plt.imshow(img_np[0,:,:])
    plt.gray()
plt.axis('off')
img_var = np_to_var(img_np).type(dtype)
d = img_np.shape[1]
out_ch = img_np.shape[0]
d_image = img_np.size





# 2. Compressed sensing using generative models

## 2.1. Load a pretrained generative model on the dataset (now: PGGAN)

In [ ]:
use_gpu = True if torch.cuda.is_available() else False

# trained on high-quality celebrity faces "celebA" dataset
# this model outputs 512 x 512 pixel images
model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
                       'PGAN', model_name='celeba',
                       pretrained=True, useGPU=use_gpu)
# this model outputs 256 x 256 pixel images
# model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
#                        'PGAN', model_name='celebAHQ-256',
#                        pretrained=True, useGPU=use_gpu)
G = model.netG
mse = torch.nn.MSELoss()


## 2.2. A=I

In [ ]:
#G0 = G(torch.zeros(1,model.config.noiseVectorDim))

def exp_lr_scheduler(optimizer, epoch, init_lr=0.005, lr_decay_epoch=100, factor=0.8):
    """Decay learning rate by a factor of 0.5 every lr_decay_epoch epochs."""
    lr = init_lr * (factor**(epoch // lr_decay_epoch))
    if epoch % lr_decay_epoch == 0:
        print('\nLR is set to {}'.format(lr))
        print('\n')
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer

def fit(G, latentDim, y, num_iter = 2000, lr_decay_epoch = 200):
    z = Variable(torch.zeros(1,latentDim), requires_grad = True)
    optimizer = optim.Adam([z])
    for i in range(num_iter):
        #################
        if lr_decay_epoch is not 0:
            optimizer = exp_lr_scheduler(optimizer, i, init_lr=0.1, lr_decay_epoch=lr_decay_epoch, factor=0.8)
            
        #################
        def closure():
            optimizer.zero_grad()           
            x = G(z)
            loss = mse(x, y)
            loss.backward(retain_graph=True)
            if i % 10 == 0:
                print('loss = {}'.format(loss))
            return loss

        loss = optimizer.step(closure)
    print('done')
    print('loss = {}'.format(loss))
    #print(z)
    return z

In [ ]:
img_var = 2*img_var - 1 #convert image to [-1,1]

z_hat = fit(G = G, latentDim = model.config.noiseVectorDim, y= img_var, num_iter = 1200, lr_decay_epoch = 400)    
    
#gen_image = (G(z_hat)+1)/2

print(gen_image)

#grid = torchvision.utils.make_grid(generated_images.clamp(min=-1, max=1), scale_each=True, normalize=True)
grid = torchvision.utils.make_grid(gen_image.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())

#print(gen_image-G0)

## 2.3. A= Gaussian 

In [ ]:
d_image = G0.numel()

f = 0.2 #compression rate
print('Compression rate is ', f)
m_image = int(f*d_image)
print('Number of measurements is ',m_image, ' for signal of length ', d_image)


# random Gaussian measurement matrix : A

A = torch.randn(m_image, d_image)
x = G0.reshape(d_image)
y = torch.matmul(A,x)
latentDim = model.config.noiseVectorDim

def fit2(G,A,y, d_image, latentDim, num_iter = 2000, lr_decay_epoch = 200):
    z = Variable(torch.randn(1,latentDim), requires_grad = True)
    
    optimizer = optim.Adam([z])
    for i in range(num_iter):
        ############
        if lr_decay_epoch is not 0:
            optimizer = exp_lr_scheduler(optimizer, i, init_lr=0.1, lr_decay_epoch = lr_decay_epoch, factor = 0.7)
        ############
        
        def closure():
            optimizer.zero_grad()
            
            x_var = G(z).reshape(d_image)
            y_var = torch.matmul(A,x_var)
            loss = mse(y_var, y)
            
            loss.backward(retain_graph = True)
            if i % lr_decay_epoch == 0:
                print('loss = {}'.format(loss))
            return loss
        
        loss = optimizer.step(closure)
    return z



In [ ]:
z = fit2(G, A, y, num_iter = 2000, lr_decay_epoch = 200)
x_hat = G(z)

#x_hat.reshape(x.size(0),x.size(1))
grid = torchvision.utils.make_grid(x_hat.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())

# 4. Compressed Sensing using Deep decoder

## 4.1. Define the network

In [ ]:
#use decoder architecture or DC GAN architecture
decodetype = 'upsample' # transposeconv / upsample


if dataset == 'mnist':
    num_channels = [25,15,10] 
elif dataset== 'celeba':    
    num_channels = [120,30,20,10] 
else:
    num_channels = [128,64,32]
    
output_depth = img_np.shape[0] # number of output channels
net = autoencodernet(num_output_channels=output_depth,num_channels_up=num_channels,need_sigmoid=True, 
                        decodetype=decodetype
                        ).type(dtype)

print("number of parameters: ", num_param(net))
if decodetype == 'upsample':
    print(net.decoder)
elif decodetype == 'transposeconv':
    print(net.convdecoder)
net_in = copy.deepcopy(net)



In [ ]:
'''if decodetype=='upsample':
    p = [x for x in net.decoder.parameters() ] #list of all weigths
elif decodetype=='transposeconv':
    p = [x for x in net.convdecoder.parameters() ] #list of all weigths

weight_decay = 0
optimizer = torch.optim.Adam(p, lr=0.001 ,weight_decay=weight_decay)'''

## 4.2. A=I using untrained network

In [ ]:
def exp_lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=500, factor=0.5):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (factor**(epoch // lr_decay_epoch))
    if epoch % lr_decay_epoch == 0:
        print('\nLearning rate is set to {}'.format(lr))
        print('\n')
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer

def fit3(net, num_channels, d_image, y, A, num_iter = 100, lr_decay_epoch = 50):
    # compute the size of (fixed) latent vector and draw it uniformly  
    totalupsample = 2**(len(num_channels)-1)
    w = np.sqrt(int(d_image/3)) # =d_image / out_channels = số chiều của mỗi cạnh ảnh
    width = int(w/(totalupsample))
    height = int(w/(totalupsample))

    shape = [1,num_channels[0], width, height]  
    print("shape of latent code B1: ", shape)

    print("initializing latent code B1...")
    net_input = Variable(torch.zeros(shape))
    net_input.data.uniform_()
    net_input.data *= 1./10

    net_input_saved = net_input.data.clone()
    noise = net_input.data.clone()

    #x_in = net(net_input.type(dtype)).data.clone() #initializing image

    # processing optimization
    if decodetype=='upsample':
        p = [x for x in net.decoder.parameters() ] #list of all weigths
    elif decodetype=='transposeconv':
        p = [x for x in net.convdecoder.parameters() ] #list of all weigths

    weight_decay = 0
    optimizer = torch.optim.Adam(p, lr=0.001 ,weight_decay=weight_decay)
    mse = torch.nn.MSELoss()
    
    for i in range(num_iter):

        #################
        if lr_decay_epoch is not 0:
            optimizer = exp_lr_scheduler(optimizer, i, init_lr=0.0001, lr_decay_epoch=lr_decay_epoch,factor=0.8)

        #################
        def closure():
            optimizer.zero_grad()           
            x_hat = net(net_input.type(dtype))
            
            y_hat = torch.matmul(A,x_hat.reshape(d_image))
            loss = mse(y_hat, y) #torch.matmul(A,x_hat)
            loss.backward()
            #mse_wrt_truth[i] = loss.data.cpu().numpy()
            return loss

        loss = optimizer.step(closure) 

        print ('Iteration %05d   Train loss %f ' % (i, loss.detach().cpu().numpy()), '\r', end='')

    return net, net_input, loss

In [ ]:
f = 0.2 #compression rate
print('Compression rate is ', f)
m_image = int(f*d_image)
print('Number of measurements is ',m_image, ' for signal of length ', d_image)


A = torch.randn(m_image, d_image)
x = img_var.reshape(d_image)
y = torch.matmul(A,x)
print(y.shape)

In [ ]:
net, net_input, loss = fit3(net, num_channels, d_image, y=y, A = A, num_iter = 2000, lr_decay_epoch=0)
x_hat = net( net_input.type(dtype) ).data.cpu().numpy()[0]

plt.imshow(x_hat.transpose(1,2,0))
plt.show()

## 4.3. A=Gaussian using untrained network

# 5. Compressed sensing using hybrid model

## 5.1. Define hybrid model

In [ ]:
#use decoder architecture or DC GAN architecture
decodetype = 'upsample' # transposeconv / upsample


if dataset == 'mnist':
    num_channels = [25,15,10] 
elif dataset== 'celeba':    
    num_channels = [120,30,20,10] 
else:
    num_channels = [128,64,32]
    
output_depth = img_np.shape[0] # number of output channels
net = autoencodernet(num_output_channels=output_depth,num_channels_up=num_channels,need_sigmoid=True, 
                        decodetype=decodetype
                        ).type(dtype)

print("number of parameters: ", num_param(net))
if decodetype == 'upsample':
    print(net.decoder)
elif decodetype == 'transposeconv':
    print(net.convdecoder)
net_in = copy.deepcopy(net)

In [ ]:
def fit4(G, net, num_channels, d_image, y, z_0, latentDim, num_iter = 200, lr_decay_epoch = 0):
    # compute the size of (fixed) latent vector and draw it uniformly  
    totalupsample = 2**(len(num_channels)-1)
    w = np.sqrt(int(d_image/3)) # =d_image / out_channels = số chiều của mỗi cạnh ảnh
    width = int(w/(totalupsample))
    height = int(w/(totalupsample))

    shape = [1,num_channels[0], width, height]  
    print("shape of latent code B1: ", shape)

    print("initializing latent code B1...")
    net_input = Variable(torch.zeros(shape))
    net_input.data.uniform_()
    net_input.data *= 1./10

    net_input_saved = net_input.data.clone()
    noise = net_input.data.clone()
    
    # collecting all trainable parameters
    alpha = Variable(torch.ones(1), requires_grad=True)
    beta = Variable(torch.zeros(1), requires_grad=True)
    z = Variable(torch.randn(1,latentDim), requires_grad = True)
    
    if decodetype=='upsample':
        p = [x for x in net.decoder.parameters() ] #list of all weigths
    elif decodetype=='transposeconv':
        p = [x for x in net.convdecoder.parameters() ] #list of all weigths
    
    
    
    weight_decay = 0
    #optimizer = torch.optim.Adam(p, lr=0.001)
    
    optimizer = torch.optim.Adam(
    [
        {"params": p, "lr": 0.001},
        {"params": alpha, "lr": 0.01},
        {"params": beta, "lr": 0.01},
        {"params": z, "lr": 0.002}
    ],
    lr=5e-4,
)
    
    mse = torch.nn.MSELoss()
    
    for i in range(num_iter):

        #################
        #if lr_decay_epoch is not 0:
        #    optimizer = exp_lr_scheduler(optimizer, i, init_lr=0.001, lr_decay_epoch=lr_decay_epoch,factor=0.8)

        #################
        def closure():
            optimizer.zero_grad()           
            x_hat = alpha*G(z) + beta*net(net_input.type(dtype))
            y_hat = x_hat
            #y_hat = torch.matmul(A,x_hat.reshape(d_image))
            loss = mse(x_hat, y) #torch.matmul(A,x_hat)
            loss.backward()
            #mse_wrt_truth[i] = loss.data.cpu().numpy()
            return loss

        loss = optimizer.step(closure) 

        print ('Iteration %05d   Train loss %f ' % (i, loss.detach().cpu().numpy()), '\r', end='')

    return net, net_input, z, alpha, beta, loss

In [ ]:
latentDim = model.config.noiseVectorDim

z0 = fit(G, latentDim, num_iter = 50, lr_decay_epoch = 0)

net, net_input, z, alpha, beta, loss = fit4(G, net, num_channels,
                                           d_image, img_var, z0,
                                           latentDim,)

In [ ]:
x_hat = alpha*G(z) + beta*net(net_input.type(dtype))
x_hat_np = x_hat.data.cpu().numpy()[0]

plt.imshow(x_hat_np.transpose(1,2,0))
plt.show()


In [ ]:
print(alpha,beta)

In [ ]:
u = torch.randn(3)
uu = 2*u -1
print(u,uu)